In [1]:
# GPU 할당확인 
import torch

print('CUDA GPU availalbe : {}'.format(torch.cuda.is_available()))
try:
    print('{} GPU(s) is(are) allocated'.format(torch.cuda.device_count()))
except:
    print('GPUs are not allocated. Current runtime is on CPU.')

CUDA GPU availalbe : True
1 GPU(s) is(are) allocated


In [2]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import random
import cv2
import copy

import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam, lr_scheduler
import torch.nn.functional as F
from efficientnet_pytorch import EfficientNet
from torch.utils.tensorboard import SummaryWriter

In [3]:
csv_dir = 'input/data/train/train.csv'
train_dir = 'input/data/train'
root_dir = 'input/data/train/images/'

class MyDataset(Dataset):
    def __init__(self, csv_file, root_dir, is_Train=True, transform=None):
        """
        Args:
            csv_file (string): csv 파일의 경로
            img_dir (string): 모든 이미지가 존재하는 디렉토리 경로
            transform (callable, optional): 샘플에 적용될 Optional transform
        """
        super().__init__()
        
        csv = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
        self.img_path = []
        path = csv['path']
        
        for person in path:
            images = [root_dir + person + '/' + image for image in os.listdir(root_dir + person) if not image[:1] == '.']
            for image in images:
                self.img_path.append(image)

        self.comb_dic = {}
        comb = [(m, g, a) for m in ['m', 'i', 'n'] for g in ['male', 'female'] for a in [0, 1, 2]]
        for i, (m, g, a) in enumerate(comb):
            self.comb_dic[(m, g, a)] = i

    def __len__(self):
        return len(self.img_path)


    def __getitem__(self, idx):
        img_name = self.img_path[idx]

        mask = img_name.split('/')[-1][:1]
        features = img_name.split('/')[-2].split('_')
        gender = features[1]
        if features[3] in ['58', '59']:
            age = 2
        else:
            age = int(features[3]) // 30

        target = self.comb_dic[(mask, gender, age)]
        
        img = Image.open(img_name)
        
        if self.transform:
            img = self.transform(img)
        
        return img, target

In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b4', num_classes=18)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [5]:
net = Net()

Loaded pretrained weights for efficientnet-b4


In [6]:
batch_size = 128
epochs = 20
lr = 3e-4

In [7]:
from torch.utils.data.dataset import random_split
from torch.utils.data.sampler import SubsetRandomSampler

transform_train = transforms.Compose([
    #transforms.Grayscale(num_output_channels=3),
#     transforms.RandomRotation(10),
#     torchvision.transforms.CenterCrop((450, 360)),
    #transforms.RandomHorizontalFlip(p=0.2),
    transforms.Resize((200,200), Image.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    #transforms.Grayscale(num_output_channels=3),
    transforms.Resize((200,200), Image.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
# random_split
# train_dataset = MyDataset(csv_dir, root_dir, is_Train=True, transform=transform_train)
# train_loader = DataLoader(train_dataset, batch_size=batch_size, pin_memory=True,
#                           num_workers=1, shuffle=True) #sampler = sampler)

# valid_dataset = MyDataset(csv_dir, root_dir, is_Train=False, transform=transform_val)
# valid_loader = DataLoader(valid_dataset, batch_size=batch_size, pin_memory=True, num_workers=1)

# sampler split
dataset = MyDataset(csv_dir, root_dir, is_Train=True, transform=transform_train)

val_idx = []
person_idx = list(range(2700))
np.random.seed(211)
np.random.shuffle(person_idx)
n_val_person = int(2700 * 0.2)
val_person_idx = person_idx[:n_val_person]

train_indices = []
val_indices = []
for i in range(2700*7):
    if i // 7 not in val_person_idx:
        train_indices.append(i)
    else:
        val_indices.append(i)


# n_val = int(len(dataset) * 0.2)
# n_train = len(dataset) - n_val
# train_dataset, valid_dataset = random_split(dataset, [len(dataset),0]) # comment when split by person
# train_indices = list(range(n_train))
# np.random.shuffle(train_indices)
train_loader = DataLoader(dataset, batch_size=batch_size, pin_memory=True, num_workers=4, sampler=train_indices)
valid_loader = DataLoader(dataset, batch_size=batch_size, pin_memory=True, num_workers=4, sampler=val_indices)

# # SubsetRandomSampler split
# dataset = MyDataset(csv_dir, root_dir, is_Train=True, transform=transform_train)

# validation_split = 0.2
# shuffle_dataset = True
# random_seed = 42
# dataset_size = len(dataset)
# indices = list(range(dataset_size))
# split = int(np.floor(validation_split * dataset_size))
# if shuffle_dataset:
#     np.random.seed(random_seed)
#     np.random.shuffle(indices)
# train_indices, val_indices = indices[split:], indices[:split]

# train_sampler = SubsetRandomSampler(train_indices)
# valid_sampler = SubsetRandomSampler(val_indices)

# # train_dataset, valid_dataset = random_split(dataset, [len(dataset),0]) # comment when split by person
# train_loader = DataLoader(dataset, batch_size=batch_size, pin_memory=True, num_workers=4, sampler=train_sampler)
# valid_loader = DataLoader(dataset, batch_size=batch_size, pin_memory=True, num_workers=4, sampler=valid_sampler)

In [8]:
from torch.optim import AdamW

In [9]:
# -- Focal Loss
# https://discuss.pytorch.org/t/is-this-a-correct-implementation-for-focal-loss-in-pytorch/43327/8
class FocalLoss(nn.Module):
    def __init__(self, weight=None,
                 gamma=0.8, reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )

In [10]:
#criterion = nn.CrossEntropyLoss()
criterion = FocalLoss()
optimizer = AdamW(net.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.2)

In [11]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [12]:
writer = SummaryWriter('model/final_experiment4')
model_name = "agefilter_focal_noval"

In [ ]:
net.cuda()
best_acc = 0.0

for epoch in range(epochs):
    print('Epoch {}/{}'.format(epoch + 1, epochs))
    print('='*12)
    
    # Train
    for iter, (img, label) in enumerate(train_loader):
        optimizer.zero_grad()
        img, label = img.type(torch.FloatTensor).cuda(), label.cuda()
        pred_logit = net(img)
        loss = criterion(pred_logit, label)
        loss.backward()
        optimizer.step()
        train_loss = loss.item()
  
        # Validation
        if (iter % 20 == 0) or (iter == len(train_loader)-1):
# val
            valid_loss, valid_acc = AverageMeter(), AverageMeter()
            
            #print("   pred: ", pred_logit.argmax(dim=-1)[:8])
            #print("   label:", label[:8])

            for img, label in valid_loader:
                img, label = img.type(torch.FloatTensor).cuda(), label.cuda()
                with torch.no_grad():
                    pred_logit = net(img)
                loss = criterion(pred_logit, label)

                # Accuracy 계산
                pred_label = torch.max(pred_logit, 1)[1]
                acc = (pred_label == label).sum() / np.float(len(img))
                valid_loss.update(loss.item(), len(img))
                valid_acc.update(acc, len(img))

            valid_loss = valid_loss.avg
            valid_acc = valid_acc.avg

            print("Iter [%3d/%3d] | Train Loss %.4f | Valid Loss %.4f | Valid Acc %.4f" %
                (iter, len(train_loader), train_loss, valid_loss, valid_acc))
# val
            
#             print("Iter [%3d/%3d] | Train Loss %.4f " %
#                 (iter, len(train_loader), train_loss))

            writer.add_scalar('training loss', train_loss, epoch * len(train_loader) + iter)
# val
            writer.add_scalar('validation loss', valid_loss, epoch * len(train_loader) + iter)
            writer.add_scalar('validation acc', valid_acc, epoch * len(train_loader) + iter)
      
            if valid_acc > best_acc:
                best_acc = valid_acc
                best_model_weights = copy.deepcopy(net.state_dict())
# val             
            best_model_weights = copy.deepcopy(net.state_dict())
            
    if epoch < epochs:
        scheduler.step()
    
    torch.save(best_model_weights, f'model/efficientnet_{model_name}_epoch{epoch + 1}.pth')
    print("BEST in epoch {} | Acc {:.4f}".format(epoch + 1, best_acc))
    
    
print('Finished training')

Epoch 1/20
Iter [  0/119] | Train Loss 2.7840 | Valid Loss 2.7552 | Valid Acc 0.0706
Iter [ 20/119] | Train Loss 2.9889 | Valid Loss 2.3982 | Valid Acc 0.2471
Iter [ 40/119] | Train Loss 1.6138 | Valid Loss 1.4591 | Valid Acc 0.4119
Iter [ 60/119] | Train Loss 3.3728 | Valid Loss 2.3437 | Valid Acc 0.2884
Iter [ 80/119] | Train Loss 1.2709 | Valid Loss 1.8966 | Valid Acc 0.2511
Iter [100/119] | Train Loss 1.6714 | Valid Loss 1.2813 | Valid Acc 0.4175
Iter [118/119] | Train Loss 0.5030 | Valid Loss 1.7793 | Valid Acc 0.3558
BEST in epoch 1 | Acc 0.4175
Epoch 2/20
Iter [  0/119] | Train Loss 3.0879 | Valid Loss 1.8208 | Valid Acc 0.3370
Iter [ 20/119] | Train Loss 1.2426 | Valid Loss 1.3097 | Valid Acc 0.3601
Iter [ 40/119] | Train Loss 1.0720 | Valid Loss 0.9697 | Valid Acc 0.5376
Iter [ 60/119] | Train Loss 1.7810 | Valid Loss 1.3439 | Valid Acc 0.4153
Iter [ 80/119] | Train Loss 0.7012 | Valid Loss 1.3166 | Valid Acc 0.4108
Iter [100/119] | Train Loss 1.2238 | Valid Loss 0.9743 | Vali

In [ ]:
# test_dir = 'input/data/eval'

# class TestDataset(Dataset):
#     def __init__(self, img_paths, transform):
#         self.img_paths = img_paths
#         self.transform = transform

#     def __getitem__(self, index):
#         image = Image.open(self.img_paths[index])

#         if self.transform:
#             image = self.transform(image)
#         return image
    
#     def __len__(self):
#         return len(self.img_paths)

In [ ]:
# # meta 데이터와 이미지 경로를 불러옵니다.
# submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
# image_dir = os.path.join(test_dir, 'images')

# # Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
# image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

# dataset = TestDataset(image_paths, transform_val)

# loader = DataLoader(
#     dataset,
#     batch_size=256,
#     shuffle=False,
#     num_workers=4
# )

# # 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
# model = Net()
# model.load_state_dict(torch.load('model/efficientnet_agefilter_focal_noval_epoch2.pth'))
# model.eval()
# model.cuda()

# # 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
# all_predictions = []
# for img in loader:
#     img = img.type(torch.FloatTensor).cuda()
#     with torch.no_grad():
#         pred_logit = model(img)
#         pred = pred_logit.argmax(dim=-1)
#         all_predictions.extend(pred.cpu().numpy())
# submission['ans'] = all_predictions

# # 제출할 파일을 저장합니다.
# submission.to_csv(os.path.join(test_dir, '4th_10percent_valset.csv'), index=False)
# print('test inference is done!')